5. Desarrollar una aplicación de software que almacene en un archivo los siguientes datos de personas: Apellido y
nombre, dirección, dni y 8 campos bivaluados, del tipo S/N o True/False, por ejemplo: estudios primarios (S/N),
estudios secundarios (S/N), estudios universitarios (S/N), tiene vivienda propia (S/N), obra social (S/N), trabaja
(S/N), etc. El programa desarrollado deberá permitir:  
a) almacenar y recuperar los datos en un archivo con campos de longitud fija (archivo “fijos.dat”).  
b) almacenar y recuperar los datos en un archivo de longitud variable (archivo “variable.dat”).  
Ingresar los mismos datos en ambos archivos para unas 20 (veinte) personas, comparar el tamaño de ambos
archivos.

Se importan las librerias


In [1]:
import struct
import os

In [2]:
# Definimos el formato fijo del registro:
# 25s -> 25 bytes (nombre)
# 30s -> 30 bytes (direccion)
# 8s  -> 8 bytes (dni)
# B   -> 1 byte (flags)
FORMATO = "25s30s8sB"
TAM_REGISTRO = struct.calcsize(FORMATO)

Se definen dos funciones set_flag, que enciende un bit en una posición determinada de un número, y get_flag, que permite comprobar si ese bit está encendido; estas funciones sirven para representar respuestas de tipo sí/no de manera compacta dentro de un solo entero. Luego, en la función alta_persona, el programa solicita datos básicos de una persona (nombre, dirección y DNI) y hace una serie de preguntas de respuesta binaria (estudios, vivienda, obra social, trabajo, auto, mascota). Por cada respuesta afirmativa, se activa un bit en la variable flags usando la función set_flag. Una vez reunida la información, se procede a guardarla en dos archivos distintos el archivo de longitud fija (fijos.dat) y en el archivo de longitud variable (variables.dat)

In [3]:
def set_flag(flags, pos):
    return flags | (1 << pos)

def get_flag(flags, pos):
    return (flags & (1 << pos)) != 0

def alta_persona():
    print("\n--- ALTA DE PERSONA ---")
    nombre = input("Apellido y nombre: ").strip()
    direccion = input("Dirección: ").strip()
    dni = input("DNI: ").strip()

    flags = 0
    preguntas = [
        "Estudios primarios (S/N): ",
        "Estudios secundarios (S/N): ",
        "Estudios universitarios (S/N): ",
        "Vivienda propia (S/N): ",
        "Obra social (S/N): ",
        "Trabaja (S/N): ",
        "Tiene auto (S/N): ",
        "Tiene mascota (S/N): "
    ]

    for i, preg in enumerate(preguntas):
        resp = input(preg).strip().upper()
        if resp == "S":
            flags = set_flag(flags, i)

    # Guardar en archivo de longitud fija
    nombre_bytes = nombre.encode("utf-8")[:30].ljust(30, b' ')
    direccion_bytes = direccion.encode("utf-8")[:50].ljust(50, b' ')
    dni_bytes = dni.encode("utf-8")[:8].ljust(8, b' ')

    registro = struct.pack(FORMATO, nombre_bytes, direccion_bytes, dni_bytes, flags)

    with open("fijos.dat", "ab") as f:
        f.write(registro)
    print("Persona guardada en archivo de longitud fija.")

    # Guardar en archivo de longitud variable
    registro = f"{nombre}|{direccion}|{dni}|{flags}\n"

    with open("variables.dat", "a", encoding="utf-8") as f:
        f.write(registro)
    print("Persona guardada en archivo de longitud variable.")

    print()

Esta función listar_personas se encarga de mostrar en pantalla los registros previamente almacenados en los archivos. Primero, solicita al usuario que elija si quiere visualizar el archivo de longitud fija, el de longitud variable o ambos. Si se selecciona el archivo de longitud fija, se abre fijos.dat en modo binario y se van leyendo los registros de tamaño fijo. Con ayuda de struct.unpack se separan los campos (nombre, dirección, DNI y flags), los textos se decodifican y se quitan los espacios de relleno, mientras que las banderas (flags) se interpretan con la función get_flag, que permite mostrar las respuestas sí/no correspondientes a estudios, vivienda, trabajo, auto y mascota. En caso de que el archivo no exista, se informa al usuario. Por otro lado, si se selecciona el archivo de longitud variable, se abre variables.dat en modo texto, se recorren las líneas y se dividen los campos usando el carácter |. A partir de esos datos se muestra la información de cada persona y también se interpretan las banderas con get_flag para reconstruir las respuestas binarias.

In [4]:

def listar_personas():
    print("\n--- LISTADO DE PERSONAS ---")

    # Preguntar qué archivo mostrar
    print("¿Qué archivo desea visualizar?")
    print("1. Archivo de longitud fija")
    print("2. Archivo de longitud variable")
    print("3. Ambos archivos")
    opcion = input("Opción (1-3): ").strip()

    if opcion in ["1", "3"]:
        print("\n=== ARCHIVO DE LONGITUD FIJA ===")
        try:
            with open("fijos.dat", "rb") as f:
                while True:
                    data = f.read(TAM_REGISTRO)
                    if not data:
                        break
                    nombre, direccion, dni, flags = struct.unpack(FORMATO, data)
                    print("Nombre:", nombre.decode("utf-8").strip())
                    print("Dirección:", direccion.decode("utf-8").strip())
                    print("DNI:", dni.decode("utf-8").strip())
                    print("Primarios:", "Sí" if get_flag(flags, 0) else "No")
                    print("Secundarios:", "Sí" if get_flag(flags, 1) else "No")
                    print("Universitarios:", "Sí" if get_flag(flags, 2) else "No")
                    print("Vivienda:", "Sí" if get_flag(flags, 3) else "No")
                    print("Obra social:", "Sí" if get_flag(flags, 4) else "No")
                    print("Trabaja:", "Sí" if get_flag(flags, 5) else "No")
                    print("Tiene auto:", "Sí" if get_flag(flags, 6) else "No")
                    print("Tiene mascota:", "Sí" if get_flag(flags, 7) else "No")
                    print("-" * 40)
        except FileNotFoundError:
            print("No hay registros en el archivo de longitud fija.")

    if opcion in ["2", "3"]:
        print("\n=== ARCHIVO DE LONGITUD VARIABLE ===")
        try:
            with open("variables.dat", "r", encoding="utf-8") as f:
                for linea in f:
                    campos = linea.strip().split('|')
                    if len(campos) >= 4:
                        nombre, direccion, dni, flags_str = campos[0], campos[1], campos[2], campos[3]
                        try:
                            flags = int(flags_str)
                        except ValueError:
                            flags = 0

                        print("Nombre:", nombre)
                        print("Dirección:", direccion)
                        print("DNI:", dni)
                        print("Primarios:", "Sí" if get_flag(flags, 0) else "No")
                        print("Secundarios:", "Sí" if get_flag(flags, 1) else "No")
                        print("Universitarios:", "Sí" if get_flag(flags, 2) else "No")
                        print("Vivienda:", "Sí" if get_flag(flags, 3) else "No")
                        print("Obra social:", "Sí" if get_flag(flags, 4) else "No")
                        print("Trabaja:", "Sí" if get_flag(flags, 5) else "No")
                        print("Tiene auto:", "Sí" if get_flag(flags, 6) else "No")
                        print("Tiene mascota:", "Sí" if get_flag(flags, 7) else "No")
                        print("-" * 40)
        except FileNotFoundError:
            print("No hay registros en el archivo de longitud variable.")

    print()

La función comparar_archivos se encarga de analizar y mostrar las diferencias entre el archivo de longitud fija (fijos.dat) y el archivo de longitud variable (variables.dat). Para hacerlo, primero obtiene el tamaño en bytes de cada archivo utilizando os.path.getsize, siempre y cuando existan; si no existen, el tamaño se toma como cero. Luego, cuenta cuántos registros contiene cada archivo: en el caso del archivo fijo, calcula la cantidad dividiendo el tamaño total entre el tamaño de un registro (TAM_REGISTRO), mientras que en el archivo variable simplemente cuenta cuántas líneas no vacías hay. Una vez obtenida esta información, muestra en pantalla un resumen con el tamaño total, la cantidad de registros y, en el caso del archivo variable, también el tamaño promedio de cada registro. Finalmente, compara los tamaños de ambos archivos e indica cuál de ellos ocupa más espacio en disco o si son iguales.

In [5]:
def comparar_archivos():
    print("\n--- COMPARACIÓN DE ARCHIVOS ---")

    try:
        # Obtener tamaños de archivos
        tamaño_fijo = os.path.getsize("fijos.dat") if os.path.exists("fijos.dat") else 0
        tamaño_variable = os.path.getsize("variables.dat") if os.path.exists("variables.dat") else 0

        # Contar registros
        registros_fijos = 0
        if os.path.exists("fijos.dat"):
            with open("fijos.dat", "rb") as f:
                f.seek(0, 2)  # Ir al final del archivo
                file_size = f.tell()
                registros_fijos = file_size // TAM_REGISTRO

        registros_variables = 0
        if os.path.exists("variables.dat"):
            with open("variables.dat", "r", encoding="utf-8") as f:
                registros_variables = sum(1 for line in f if line.strip())

        # Mostrar comparación
        print("COMPARACIÓN DE ARCHIVOS")
        print("=" * 50)
        print()

        print(f"Archivo de longitud fija (fijos.dat):")
        print(f"  - Tamaño: {tamaño_fijo} bytes")
        print(f"  - Registros: {registros_fijos}")
        print(f"  - Tamaño por registro: {TAM_REGISTRO} bytes")
        print()

        print(f"Archivo de longitud variable (variables.dat):")
        print(f"  - Tamaño: {tamaño_variable} bytes")
        print(f"  - Registros: {registros_variables}")
        if registros_variables > 0:
            tamaño_promedio = tamaño_variable / registros_variables
            print(f"  - Tamaño promedio por registro: {tamaño_promedio:.2f} bytes")
        else:
            print(f"  - Tamaño promedio por registro: 0 bytes")
        print()

        # Comparación
        if tamaño_fijo > 0 and tamaño_variable > 0:
            diferencia = tamaño_fijo - tamaño_variable

            print("COMPARACIÓN:")
            if diferencia > 0:
                print(f"  - El archivo de longitud fija es {diferencia} bytes más grande")
            elif diferencia < 0:
                print(f"  - El archivo de longitud variable es {-diferencia} bytes más grande")
            else:
                print("  - Ambos archivos tienen el mismo tamaño")

    except Exception as e:
        print(f"Error al comparar archivos: {str(e)}")

    print()

La función generar_registros_prueba permite generar y cargar 19 registros a los archivos fijos.dat y variables.dat

In [6]:
def generar_registros_prueba():
    print("\n--- GENERAR REGISTROS DE PRUEBA ---")

    # Datos de prueba
    nombres = ["García Juan", "López María", "Pérez Carlos", "Rodríguez Ana", "González Luis",
              "Martínez Laura", "Fernández Diego", "Sánchez Marta", "Romero Pablo", "Álvarez Sofía",
              "Torres Miguel", "Domínguez Elena", "Vázquez Andrés", "Ramos Isabel", "Gil Francisco",
              "Serrano Carmen", "Blanco José", "Molina Patricia", "Morales Antonio"]

    direcciones = ["Calle Mayor 123", "Avenida Libertad 45", "Plaza España 6", "Calle Sol 78", "Avenida Central 9",
                  "Calle Luna 12", "Paseo del Parque 34", "Calle Flor 56", "Avenida Norte 78", "Calle Sur 90",
                  "Calle Este 11", "Calle Oeste 22", "Avenida Este 33", "Avenida Oeste 44", "Calle Real 55",
                  "Avenida Rey 66", "Calle Príncipe 77", "Avenida Princesa 88", "Calle Duque 99"]

    dnis = ["12345678", "23456789", "34567890", "45678901", "56789012", "67890123", "78901234", "89012345",
           "90123456", "01234567", "11223344", "22334455", "33445566", "44556677", "55667788",
           "66778899", "77889900", "88990011", "99001122"]

    # Generar 19 registros
    for i in range(19):
        nombre = nombres[i]
        direccion = direcciones[i]
        dni = dnis[i]

        # Configurar flags (alternando entre diferentes combinaciones)
        flags = 0
        if i % 2 == 0:
            flags = set_flag(flags, 0)  # Primarios
        if i % 3 == 0:
            flags = set_flag(flags, 1)  # Secundarios
        if i % 4 == 0:
            flags = set_flag(flags, 2)  # Universitarios
        if i % 5 == 0:
            flags = set_flag(flags, 3)  # Vivienda
        if i % 2 == 1:
            flags = set_flag(flags, 4)  # Obra social
        if i % 3 == 1:
            flags = set_flag(flags, 5)  # Trabaja
        if i % 4 == 1:
            flags = set_flag(flags, 6)  # Tiene auto
        if i % 5 == 1:
            flags = set_flag(flags, 7)  # Tiene mascota

        # Guardar en archivo de longitud fija
        nombre_bytes = nombre.encode("utf-8")[:30].ljust(30, b' ')
        direccion_bytes = direccion.encode("utf-8")[:50].ljust(50, b' ')
        dni_bytes = dni.encode("utf-8")[:8].ljust(8, b' ')
        registro_fijo = struct.pack(FORMATO, nombre_bytes, direccion_bytes, dni_bytes, flags)

        with open("fijos.dat", "ab") as f:
            f.write(registro_fijo)

        # Guardar en archivo de longitud variable
        registro_variable = f"{nombre}|{direccion}|{dni}|{flags}\n"
        with open("variables.dat", "a", encoding="utf-8") as f:
            f.write(registro_variable)

    print("Se han generado 19 registros de prueba en ambos archivos.")
    print()

En la funcion main se implementa un menú principal con las opciones
1.Alta persona
2.Listar personas
3.Comparar archivos
4.Generar 19 archivos prueba.
0.Salir

In [7]:
def main():
    while True:
        print("\n--- MENÚ PRINCIPAL ---")
        print("1. Alta persona")
        print("2. Listar personas")
        print("3. Comparar archivos")
        print("4. Generar 19 registros de prueba")
        print("0. Salir")
        opcion = input("Opción: ").strip()

        if opcion == "1":
            alta_persona()
        elif opcion == "2":
            listar_personas()
        elif opcion == "3":
            comparar_archivos()
        elif opcion == "4":
            generar_registros_prueba()
        elif opcion == "0":
            break
        else:
            print("Opción inválida.")

if __name__ == "__main__":
    main()


--- MENÚ PRINCIPAL ---
1. Alta persona
2. Listar personas
3. Comparar archivos
4. Generar 19 registros de prueba
0. Salir
Opción: 1

--- ALTA DE PERSONA ---
Apellido y nombre: Ortega Raquel
Dirección: Avenida Duquesa 100
DNI: 00112233
Estudios primarios (S/N): S
Estudios secundarios (S/N): S
Estudios universitarios (S/N): S
Vivienda propia (S/N): S
Obra social (S/N): N
Trabaja (S/N): S
Tiene auto (S/N): S
Tiene mascota (S/N): N
Persona guardada en archivo de longitud fija.
Persona guardada en archivo de longitud variable.


--- MENÚ PRINCIPAL ---
1. Alta persona
2. Listar personas
3. Comparar archivos
4. Generar 19 registros de prueba
0. Salir
Opción: 2

--- LISTADO DE PERSONAS ---
¿Qué archivo desea visualizar?
1. Archivo de longitud fija
2. Archivo de longitud variable
3. Ambos archivos
Opción (1-3): 3

=== ARCHIVO DE LONGITUD FIJA ===
Nombre: Ortega Raquel
Dirección: Avenida Duquesa 100
DNI: 00112233
Primarios: Sí
Secundarios: Sí
Universitarios: Sí
Vivienda: Sí
Obra social: No
Trab

Se realizó el alta de una persona manualmente, ingresando los siguientes datos:
Apellido y nombre: Ortega Raquel,
Dirección: Avenida Duquesa 100,
DNI: 00112233,
Estudios primarios: Sí,
Estudios secundarios: Sí,
Estudios universitarios: Sí,
Vivienda propia: Sí,
Obra social: No,
Trabaja: Sí,
Tiene auto: Sí,
Tiene mascota: No.

Posteriormente se verificó que esta persona estuviera correctamente registrada tanto en el archivo fijos.dat como en el archivo variables.dat. Se realizó la comparación de archivos para la persona cargada, observándose que el archivo de longitud fija era 17 bytes más grande que el de longitud variable con un solo registro cargado.

Luego se cargaron 19 personas más para completar un total de 20 registros, según lo solicitado en la consigna, y se volvió a comparar ambos archivos con todos los registros cargados. Como resultado, el archivo de longitud fija (fijos.dat) quedó con un tamaño de 1280 bytes, mientras que el archivo de longitud variable (variables.dat) ocupó 864 bytes, lo que indica que el archivo fijo era 416 bytes más grande que el variable.
